# Anno 1800: Maximize population by adjusting skyscraper levels

This is an interactive document consisting of text parts (like this one) and code parts (with a greyish background). Click in the next line and press **Shift + Enter** to execute the code.

In [ ]:
import importlib
if importlib.util.find_spec("pulp") is None :
    %pip install pulp
if importlib.util.find_spec("watchdog") is None :
    %pip install watchdog

The next code block is collapsed because it is very long. To execute it without expanding, click here and then twice shift + enter. That way you "execute" this markdown cell and the following code cell.

In [ ]:
import numpy as np
from scipy import ndimage
import pulp
from pulp import *
from collections import deque
from queue import PriorityQueue

import copy
import json
import math
import pathlib
import PIL.Image
import IPython.display
import shutil
import time
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

# for logging
from os import dup, dup2, close
import tempfile

one = np.array([1,1])

class Level :
    def __init__(self, index, level, template, radius, color, pop, stores, stores_th, th, max_residents_th) :
        self.index = index
        self.level = level
        self.template = template
        self.radius = radius
        self.color = color
        self.pop = pop
        self.stores = stores
        self.stores_th = stores_th
        self.th = th
        self.max_residents_th = max_residents_th

panoramaGain = 25
r_th = 20

def pos(obj) :
    return np.array([int(c) for c in obj['Position'].split(',')]) 

def size(obj) :
    return np.array([int(c) for c in obj['Size'].split(',')])

#simple image scaling to (nR x nC) size
def scale(im, nR, nC):
    nR0 = len(im)     # source number of self.rows 
    nC0 = len(im[0])  # source number of columns 
    return [[ im[int(nR0 * r / nR)][int(nC0 * c / nC)]  
             for c in range(nC)] for r in range(nR)]

def generate_houses(block_length, rows, cols):
    size = 3
    house = {"Identifier":"Farmer Residence","Label":"","Position":"1,1","Size":"3,3","Icon":"A7_resident","Template":"Farmer Residence","Color":{"A":255,"R":161,"G":234,"B":234},"Borderless":False,"Road":False,"Radius":0.0,"InfluenceRange":0.0,"PavedStreet":True,"BlockedAreaLength":0.0,"BlockedAreaWidth":0.0,"Direction":"Down"}
    houses = []  
    
    
    for bx in range(0,rows):
        for by in range(0,cols):
            for hx in range(0,2) :
                for hy in range(0, block_length) :
                    x = bx * (2 * size + 1) + size * hx + 1
                    y = by * (block_length * size + 1) + size * hy + 1
                    obj = copy.deepcopy(house)
                    obj["Position"] = "{},{}".format(x,y)
                    houses.append(obj)                        
                    
    
    return {
        "FileVersion":4,
        "LayoutVersion":"1.0.0.0",
        "Objects": houses
    }

class House :
    def __init__(self, obj, ID, tl = None) :
        self.obj = obj
        self.tl = tl
        if self.tl is None :
            self.tl = pos(obj)
        self.size = np.array([3,3])
        self.center = self.tl + one
        self.neighbors = []
        self.ID = ID
        
        self.stores = [False, False, False]
        self.electricity = False
        self.th = False
        
        self.var_levels = [LpVariable("HL_{}_{}".format(ID, l.level), cat='Binary') for l in levels]
        self.level = 4
        self.panorama_effect = 0
        
        for l in levels :
            if l.template == obj["Identifier"] :
                self.level = l.level
                break

        
    def __lt__(self,other):
        return self.center[1] < other.center[1] if self.center[0] == other.center[0] else self.center[0] < other.center[0]
        
    def dist(self, h) :
        return np.linalg.norm(self.center - h.center)
    
    def is_gap(self) :
        return not self.obj.get("Label") is None and (self.obj["Label"] == "0" or self.obj["Label"] == "-")
    
    def calculate_panorama(self, level = None) :
        if level is None :
            level = self.level
        
        support = level
            
        for n in self.neighbors :
            if levels[level-1].radius < self.dist(n) :
                continue
                
            if n.level < level :
                support += 1
            else :
                support -= 1
                
        return max(0,min(5,support))      
           
    
    def get_residents(self, level = None, panorama_effect = None) :
        if level is None:
            level = levels[self.level - 1]
        if panorama_effect is None :
            panorama_effect = self.panorama_effect
            
        pop = level.pop
        for s in range(len(level.stores)) :
            if self.stores[s] :
                s_pop = level.stores[s]
                if len(level.stores_th) > s and self.th :
                    s_pop += level.stores_th[s]
                    
                pop += int(self.stores[s] * s_pop)
        
        pGain = panoramaGain
        
        if self.th :
            pop += level.th
        if not self.electricity :
            pop -= 8
            pGain -= 5
                
        return pop + panorama_effect * pGain
    
    def fix_level(self, level) :
        self.var_levels[level - 1].setInitialValue(1)
        self.var_levels[level - 1].fixValue()
        
    def gen_object(self) : 
        l = levels[self.level - 1]
        return {"Identifier":l.template,
                "Label":str(self.get_residents()),
                "Position": self.obj["Position"],
                "Size":self.obj["Size"],
                "Icon":"A7_panorama_buff_0" + str(self.panorama_effect),
                "Template":"Farmer Residence",
                "Color":l.color,
                "Borderless":False,
                "Road":False,
                "Radius":l.radius,
                "InfluenceRange":0.0,
                "PavedStreet":True,
                "BlockedAreaLength":0.0,
                "BlockedAreaWidth":0.0,
                "Direction":"Down"}


class Layout :
    def __init__(self, ad_json) :
        r_max = math.ceil(levels[-1].radius)
        
        self.json = ad_json
        
        self.tl = np.array([99999999, 999999999])
        self.br = np.array([-99999999, -999999999])
        one = np.array([1,1])
        for obj in ad_json['Objects'] :
            self.tl = np.minimum.reduce([self.tl, pos(obj)])
            self.br = np.maximum.reduce([self.br, pos(obj) + size(obj)])
         
        #print(self.tl, self.br)
        
        self.tl = self.tl - r_th * one
        self.br = self.br + r_th * one
        
        dim = self.br - self.tl
        self.rows = dim[1]
        self.cols = dim[0]
        self.area = np.empty(shape=dim, dtype=object)
        self.streets = np.zeros(shape=dim, dtype=int)
        
        self.clusters = []
        self.cluster_gaps = set()
        
        #print(self.tl, self.br, dim)
        
        self.houses = []
        for obj in ad_json['Objects'] :
            p = pos(obj) - self.tl
            if self.is_street(obj) :
                self.streets[p[0],p[1]] = 1
            
            if not self.is_house(obj) :
                continue
                
            self.houses.append(House(obj, len(self.houses) + 1, p))
            h = self.houses[-1]  
            self.area[h.center[0], h.center[1]] = h
        
        # compute neighbouring houses that may affect panorama
        for h in self.houses :    
            for x in range(h.center[0] - r_max, h.center[0] + r_max) :
                for y in range(h.center[1] - r_max, h.center[1] + r_max) :
                    h_ = self.area[x,y]
                    if h_ is None or h == h_:
                        continue
                        
                    if(h.dist(h_) <= r_max) :
                        h.neighbors.append(h_)
        
        
        for h in self.houses :
            h.panorama_effect = h.calculate_panorama()
            
            for x in range(h.tl[0], h.tl[0] + h.size[0]) :
                for y in range(h.tl[1], h.tl[1] + h.size[1]) :
                    self.area[x][y] = h
                    
        # compute clusters           
        for h in self.houses : 
            if h.is_gap() :
                self.cluster_gaps.add(h)
                continue
            
            in_cluster = False
            for c in self.clusters:
                if h in c :
                    in_cluster = True
                    break
            
            if in_cluster :
                continue
            
            if len(h.neighbors) == 0:
                h.level = 5
                continue

            self.clusters.append(set())
            c = self.clusters[-1]
            q = deque([h])

            while len(q) :
                n = q.pop()

                for m in n.neighbors :
                    if m in c :
                        continue

                    if not m.is_gap() :
                        q.append(m)
                        c.add(m)        
        
        # compute stores, townhalls and electricity
        for obj in ad_json['Objects'] :            
            
            if self.is_powerplant(obj) :
                #print("P", pos(obj), obj["InfluenceRange"])
                self.mark_in_range(obj, obj["InfluenceRange"] if not obj.get("InfluenceRange") is None else 60, 0)
            
            s = self.get_store_index(obj)
            if s > 0 :
                self.mark_in_range(obj, 63.667, s)
            
            if self.is_th(obj):
                #print("TH", pos(obj))
                p = pos(obj) - self.tl
                center = p + np.array([1.5, 1.5])
                for x in range(p[0] - r_th, p[0] + r_th+2) :
                    for y in range(p[1] - r_th, p[1] + r_th+2) :
                        h = self.area[x][y]
                        
                        if h is None:
                            continue
                            
                     
                        if np.linalg.norm(center - h.center) <= r_th :
                            h.th = True
        
            
        
    def is_house(self, obj) :
        if obj.get("Identifier") is None or obj.get("Template") is None:
            return False
        
        if obj.get("Identifier") == "Scholar_Residence" :
            return False
        
        return "residence" in obj["Identifier"].lower() or "skyscraper" in obj["Template"].lower()
    
    def is_street(self, obj):
        return obj.get("Road")
    
    def get_store_index(self, obj):
        if obj.get("Identifier") is None or obj.get("Template") is None:
            return 0
        
        if "DepartmentStore" in obj["Identifier"] or "DepartmentStore" in obj["Template"]:
            return 1
        if "FurnitureStore" in obj["Identifier"] or "FurnitureStore" in obj["Template"]:
            return 2
        if "Pharmacy" in obj["Identifier"] or "Pharmacy" in obj["Template"]:
            return 3
        return 0
    
    def is_powerplant(self, obj):
        return not obj.get("Icon") is None and "A7_electric_works" in obj["Icon"]
    
    def is_th(self, obj):
        return not obj.get("Icon") is None and "A7_townhall" in obj.get("Icon")
    
    def draw(self, area) :
        a = np.uint8(np.clip(scale(np.transpose(area)*50,2*self.cols,2*self.rows) , 0, 255))
        image_data = io.BytesIO()
        PIL.Image.fromarray(a).save(image_data, "png")
        IPython.display.display(IPython.display.Image(data=image_data.getvalue()))
    
    def mark_in_range(self, obj, r, update_mode) :
        r += 1
        n_tiles = []
        c_tiles = []
        p = pos(obj) - self.tl
        s = size(obj)
        streets = copy.deepcopy(self.streets)
        
        for x in range(p[0], p[0] + s[0]) :
            for y in range(p[1], p[1] + s[1]) :
                if x == p[0] or y == p[1] or x == p[0] + s[0] - 1 or y == p[1] + s[1] - 1 :
                    n_tiles.append(np.array([x,y]))
                    streets[x,y] = 3
        
        
        at = lambda container, pos : container[pos[0]][pos[1]]
        
        while r > 0 :
            c_tiles = n_tiles
            n_tiles = []
           
            r -= 1
           
            for center in c_tiles:
                for direction in [np.array([1,0]),np.array([0,1]),np.array([-1,0]),np.array([0,-1])] :
                    n = center + direction
                    neighbor = at(self.area, n)
                      
                    if not neighbor is None and isinstance(neighbor, House) :
                        if update_mode == 0 :
                            neighbor.electricity = True
                        else :
                            neighbor.stores[update_mode - 1] = max(neighbor.stores[update_mode - 1], True if r >= 1 else r)
                    
                    elif at(streets, n) == 1:
                        n_tiles.append(n)
                        
                        if r >= 1:
                            streets[n[0], n[1]] = 2
            
            
        
        #print(obj["Icon"], p)
        #self.draw(streets)


class GurobiWatcher:
    #logs = pd.DataFrame(columns = ["Time", "Best Found", "Upper Bound"], dtype=float)
    log_path = None
    
    def __init__(self, log_path):
        GurobiWatcher.log_path = pathlib.Path(log_path).resolve()
        self.observer = Observer()
        self.stopped = False
  
    def run(self):
        event_handler = Handler()
        self.observer.schedule(event_handler, str(self.log_path.parent), recursive = False)
        self.observer.start()

    def stop(self):
        self.stopped = True
        self.observer.stop()
        self.observer.join()

  
  
class Handler(FileSystemEventHandler):

    @staticmethod
    def on_any_event(event):
        if event.is_directory:
            return None
  
        if event.event_type == 'modified' and pathlib.Path(event.src_path) == GurobiWatcher.log_path:
            try:
                with open(str(GurobiWatcher.log_path), "r") as f :
                    lines = f.read().splitlines()
                
                if len(lines) == 0 or len(lines[-1]) == 0:
                    return
                
                tokens = re.split(r'\s+',lines[-1])
                match = re.match(r'(\d+)s', tokens[-1])
                if match is None:
                    return None
                
                #The trailing tabulators ensure that replacing the line leaves no old letters
                print("Elapsed : {:.2f}".format(int(match[1])/60), " min\tBest Found :", float(tokens[-5]), "\tUpper Bound :", float(tokens[-4]), "\t\t\t\t", end="\r")
            except:
                pass

            
        
class LPLevels : 
    def __init__(self, layout, houses = None,  min_level = 4, log_path = None, full_supply = False) :
        self.layout = layout
        self.houses = self.layout.houses if houses is None else houses
        self.prob = LpProblem("Skyscraperlevels", LpMaximize)
        
        self.weights_profit = {}
        self.min_level = min_level
        
        weights_profit = self.weights_profit
        prob = self.prob
        
        for h in self.houses :
            for l in range(min_level - 1) :
                if full_supply and sum(h.stores) < 3:
                    if not h.stores[0] == True:
                        h.fix_level(1)
                    elif not h.stores[1] == True:
                        h.fix_level(2)
                    elif not h.stores[2] == True:
                        h.fix_level(4)
                else:                    
                    h.var_levels[l].setInitialValue(0)
                    h.var_levels[l].fixValue()
                    
                h.var_panorama = [LpVariable("HP_{}_{}".format(h.ID, l.level), cat='Integer') for l in levels]
                
        #layout.houses[2].fix_level(4)
               
                
        for h in self.houses :
            prob += 1 == sum(h.var_levels[l.index] for l in levels) 
            neighborhood = []
                
            for l in levels :
                T = []
                S = []
                L = h.var_levels[l.index]
                P = h.var_panorama[l.index]
                
                pGain = panoramaGain if h.electricity else panoramaGain - 5
                weights_profit[L] = h.get_residents(l,0)
                
                for h_ in h.neighbors :
                    if(h.dist(h_) > l.radius) :
                        continue
                        
                    for l_ in levels :
                        if(l.level > l_.level) :
                            S.append(h_.var_levels[l_.index])
                        else :
                            T.append(h_.var_levels[l_.index])
                            
                #neighborhood.append((len(S), len(T)))
                        
                  

                T_ = LpVariable("T'_{}_{}".format(h.ID, l.level), cat='Integer')
                prob += T_ <= 1 + 0.01 * (sum(t for t in T) - (sum(s for s in S)) - l.level)
                prob += P <= 5 * L
                prob += P <= l.level - sum(t for t in T) + 100 * T_ + sum(s for s in S)
                prob += P <= 5 * (1 - T_)
                weights_profit[P] = pGain

                                        
            #print(h.center, neighborhood)
 
        prob.objective = LpAffineExpression(weights_profit)
    
        if time_limit is None:
            limit = max(120, len(self.houses) / 3)
        else :
            limit = max(120, time_limit)
        
        delete_log = False
        if log_path == None :
            delete_log = True
            p = os.environ['TEMP']
            if p is None:
                log_path = pathlib.Path(os.getcwd()) / "temp_files/solver.log"
            else:
                log_path = pathlib.Path(p) / "skyscraper_levels/solver.log"
        else:
            log_path = pathlib.Path(log_path).resolve()
        
        log_path.parent.mkdir(parents=True, exist_ok=True)
        with open(log_path, 'w') as fp:
            pass
                
        w = GurobiWatcher(log_path)
        w.run()
        self.status = prob.solve(apis.GUROBI_CMD(timeLimit = limit, gapAbs = 1,logPath = str(log_path)))
        w.stop()
        
        if delete_log:
            shutil.rmtree(log_path.parent)
        
        self.level_count = [0 for l in levels]
        self.panorama_count = [0 for p in range(6)]
        for h in self.houses :
            for l in range(len(h.var_levels)) :
                if value(h.var_levels[l]) < 0.01 :
                    continue
                    
                self.level_count[l] += 1
                h.level = levels[l].level
                   
                P = h.var_panorama[l]

                self.panorama_count[int(value(P))] += 1
                h.panorama_effect = int(value(P))
               
        
        total = int(round(value(prob.objective)))
        print("Total population:", total, "| per House:", total / len(self.houses), "| Skyscraper levels:", self.level_count, "| Skyscraper panorama effects:", self.panorama_count)

    def get_summary(self) :  
        min_level = self.min_level
        summary = [{
            "Level": str(l) if l <= 5 else "Total",
            "Residences": 0,
            "Residents": 0,
            "per House": 0,
            "Max. Residents": 0, 
            "Residences in TH": 0,
            "Max. Residents in TH": 0,
            "Panorama": 0,
            "Store Coverage": [0 for i in range(3)],
            "Department": 0,
            "Furniture": 0,
            "Drug": 0,
        } for l in range(min_level,7)]
        
        for h in self.layout.houses:
            l = levels[h.level - 1]
            max_residents = 50 + h.level * 25 + h.panorama_effect * panoramaGain + sum(l.stores)

            # level summary
            s = summary[h.level - min_level]

            if h.th :
                max_residents += l.max_residents_th
                s["Residences in TH"] += 1
                s["Max. Residents in TH"] += max_residents

            s["Residences"] += 1
            s["Residents"] += h.get_residents()
            s["Max. Residents"] += max_residents
            s["Panorama"] += h.panorama_effect
            for st in range(3) :
                s["Store Coverage"][st] += 1 if h.stores[st] else 0

            #total summary
            s = summary[-1]
            if h.th :
                s["Residences in TH"] += 1
                s["Max. Residents in TH"] += max_residents

            s["Residences"] += 1
            s["Residents"] += h.get_residents()
            s["Max. Residents"] += max_residents
            s["Panorama"] += h.panorama_effect
            for st in range(3) :
                s["Store Coverage"][st] += 1 if h.stores[st] else 0

        for s in summary :
            s["per House"] = "{:.5}".format(s["Residents"] / s["Residences"])
            s["Residences in TH"] = "{:.2%}".format(s["Residences in TH"] / s["Residences"])
            s["Max. Residents in TH"] = "{:.2%}".format(s["Max. Residents in TH"] / s["Max. Residents"])
            s["Panorama"] = "{:.2}".format(s["Panorama"] / s["Residences"])
            s["Department"] = "{:.2%}".format(s["Store Coverage"][0] / s["Residences"])
            s["Furniture"] = "{:.2%}".format(s["Store Coverage"][1] / s["Residences"])
            s["Drug"] = "{:.2%}".format(s["Store Coverage"][2] / s["Residences"])
            s["Store Coverage"] = ""
            
        return summary
            
        
    def save(self, path) :
          
        src_json = self.layout.json
        ad_json = {"Objects":[]}
        
        for k in src_json.keys() :
            if not k == "Objects":
                ad_json[k] = src_json[k]
                
        for obj in src_json["Objects"] :
            if not self.layout.is_house(obj) :
                ad_json["Objects"].append(obj)
                
        for h in self.layout.houses:
            ad_json["Objects"].append(h.gen_object())
            
        summary = self.get_summary()
        
        anchor = np.array([self.layout.cols,1])
        for l in levels :
            p = anchor + l.index * np.array([3,0])
            ad_json["Objects"].append({"Identifier":"Legend","Label":"Level " + str(l.level),"Position":"{},{}".format(p[0],p[1]),"Size":"3,3","Icon":"A7_dlc_high_life_256","Template":"","Color":l.color,"Borderless":False,"Road":False,"Radius":0.0,"InfluenceRange":0.0,"PavedStreet":True,"BlockedAreaLength":0.0,"BlockedAreaWidth":0.0,"Direction":"Down"})

        anchor[1] += 4
        label = ""
        label_height = 2*len(summary[0].keys())
        label_width = 17
        for k in summary[0].keys() :
            label += k + "\n"

        ad_json["Objects"].append({"Identifier":"Legend","Label":label,"Position":"{},{}".format(anchor[0],anchor[1]),"Size":"{},{}".format(label_width,label_height),"Icon":None,"Template":"","Color":{"A": 255, "R": 255, "G": 255, "B": 255},"Borderless":True,"Road":False,"Radius":0.0,"InfluenceRange":0.0,"PavedStreet":True,"BlockedAreaLength":0.0,"BlockedAreaWidth":0.0,"Direction":"Down"})
        anchor[0] += label_width


        for s in summary :
            label = ""
            label_height = 2*len(s)
            label_width = 8
            for v in s.values() :
                label += str(v) + "\n"

            ad_json["Objects"].append({"Identifier":"Legend","Label":label,"Position":"{},{}".format(anchor[0],anchor[1]),"Size":"{},{}".format(label_width,label_height),"Icon":None,"Template":"","Color":{"A": 255, "R": 255, "G": 255, "B": 255},"Borderless":True,"Road":False,"Radius":0.0,"InfluenceRange":0.0,"PavedStreet":True,"BlockedAreaLength":0.0,"BlockedAreaWidth":0.0,"Direction":"Down"})
            anchor[0] += label_width
        
        with open(path, "w") as f :
            json.dump(ad_json, f)

First, you need to specify some settings:

In [ ]:
time_limit = 120 # Uses 1 second per 3 houses. You can manually set a limit to get better results, e.g enter (without quotes) "time_limit = 3600" for one hour (time_limit is specified in seconds)
anno_designer_file_name = r"<path-to-file>.ad" # Use an absolute path if the file does not reside in the same directory as this application

Only execute one of the following cells depending on your setup:

In [ ]:
# Equipped items: Papal paper, Blue Skies Maid
# Full supply
levels = [
    Level(index = 0, level = 1, template="A7_residence_SkyScraper_5lvl1", radius = 4, color = {"A": 255, "R": 3, "G": 94, "B": 94}, pop = 50, stores =[25], stores_th = [5], th = 15, max_residents_th = 15+5),
    Level(index = 1, level = 2, template="A7_residence_SkyScraper_5lvl2", radius = 4.25, color = {"A": 255, "R": 0, "G": 128, "B": 128}, pop = 75, stores =[25], stores_th = [5], th = 15 + 5, max_residents_th = 15+10),
    Level(index = 2, level = 3, template="A7_residence_SkyScraper_5lvl3", radius = 5, color = {"A": 255, "R": 68, "G": 166, "B": 166}, pop = 90, stores =[25,10], stores_th = [5,5], th = 15 + 10, max_residents_th = 15+20),
    Level(index = 3, level = 4, template="A7_residence_SkyScraper_5lvl4", radius = 6, color = {"A": 255, "R": 105, "G": 196, "B": 196}, pop = 115, stores =[25,10], stores_th = [5,5], th = 15 + 20, max_residents_th = 15+30), #Papal paper,  Blue Skies Maid
    Level(index = 4, level = 5, template="A7_residence_SkyScraper_5lvl5", radius = 6.75, color = {"A": 255, "R": 161, "G": 234, "B": 234}, pop = 135, stores =[25,10,5], stores_th = [5,5,5], th = 15 + 30, max_residents_th = 15+45), #Papal paper,  Blue Skies Maid
]

In [ ]:
# Equipped items: Papal paper, Saint D'Artois, Blue Skies Maid, Blue Skies, Blue Skies Delivery Service
# Do not supply Billiard tables and toys
levels = [
    Level(index = 0, level = 1, template="A7_residence_SkyScraper_5lvl1", radius = 4, color = {"A": 255, "R": 3, "G": 94, "B": 94}, pop = 50, stores =[25], stores_th = [5], th = 15 + 10, max_residents_th = 15+10+5),
    Level(index = 1, level = 2, template="A7_residence_SkyScraper_5lvl2", radius = 4.25, color = {"A": 255, "R": 0, "G": 128, "B": 128}, pop = 75, stores =[25], stores_th = [5], th = 15 + 10 + 5, max_residents_th = 15+10+10),
    Level(index = 2, level = 3, template="A7_residence_SkyScraper_5lvl3", radius = 5, color = {"A": 255, "R": 68, "G": 166, "B": 166}, pop = 90, stores =[25,10], stores_th = [5,5], th = 15 + 10 + 10, max_residents_th = 15+10+20),
    Level(index = 3, level = 4, template="A7_residence_SkyScraper_5lvl4", radius = 6, color = {"A": 255, "R": 105, "G": 196, "B": 196}, pop = 100, stores =[25,10], stores_th = [5,5+15+5], th = 15 + 10 + 15, max_residents_th = 15+10+30), #Papal paper, Saint D'Artois, Blue Skies Maid
    Level(index = 4, level = 5, template="A7_residence_SkyScraper_5lvl5", radius = 6.75, color = {"A": 255, "R": 161, "G": 234, "B": 234}, pop = 100, stores =[25,10,5], stores_th = [5,5+15+5,5+5+5], th = 15 + 10 + 15, max_residents_th = 15+10+45), #Papal paper, Saint D'Artois, Blue Skies Maid
]

Now run the optimization, print some stats and save it to another Anno Designer file. This will take some time, so be patient!

In [ ]:
with open(anno_designer_file_name, "r") as f :
    layout = Layout(json.loads(f.read())) 

p = pathlib.Path(anno_designer_file_name)
    
print("Residences:", len(layout.houses), "| Max computation time:", len(layout.houses) / 3 / 60 if time_limit is None else time_limit / 60, "min")

opt = LPLevels(layout, full_supply=True)
opt.save(str(p.parent / (p.stem + "_opt.ad")))